In [82]:
from pypdf import PdfReader
from pypdf.errors import PdfReadError
from transformers import pipeline
import pandas as pd
import torch
import sentencepiece
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk


In [31]:
#nltk.download('punkt') # only uncomment if not already downloaded
#nltk.download('stopwords')

In [87]:
ArticleToSummarize = "pdf_database\pdf1.pdf"

def pdfTE(pdfFile): # Function for text extraction from PDF.
    with open(pdfFile, "rb") as file: # Read in binary to handle breakline statements better (\n)
        pdfReader = PdfReader(file)
        for page in pdfReader.pages:
            yield page.extract_text() # Use of generator as keeping the whole article in the memory results in memory error.
            
def articleC(pdfFile): # Function to return the article in one string
    textCombiner = pdfTE(pdfFile) 
    textCombined = ""
    for text in textCombiner: # Loop over generator object to sum text of pages into one string
        textCombined += text
    return textCombined

def sumArticle1(pdfFile): # Function to summarize article per page
    if pdfFile.endswith(".pdf"): # First check if file extension is ".pdf" format.
        try:
            summarizer = pipeline("summarization", model="pszemraj/led-large-book-summary") # Model used from the huggingface hub (https://huggingface.co/pszemraj/led-large-book-summary)
            for text in pdfTE(pdfFile): # Iterate over generator
                summarizedPage = summarizer(text,max_length=100, min_length=50, do_sample=True)
                print(summarizedPage, end="\n")
                print()
        except PdfReadError as pre: # Second check if file extension is ".pdf" format when first check fails.
            return f"Oops! a PDF Read Error {pre} happened. Please retry the task once the issue has been resolved."
        except OSError as os: # Exception handling of OS errors, as PdfReadError doesn't catch all file extension errors.
            return f"Oops! an OS Error {os} happened. Please retry the task once the issue has been resolved."
    else:
        return f"Oops! The task failed. Please retry the task once the issue has been resolved." # Message if program fails to execute

def sumArticle2(pdfFile): # Function to summarize article as a whole
    if pdfFile.endswith(".pdf"): # First check if file extension is ".pdf" format.
        try:
            summarizer = pipeline("summarization", model="pszemraj/led-large-book-summary") # Model used from the huggingface hub (https://huggingface.co/pszemraj/led-large-book-summary)
            articleCombined = articleC(pdfFile) # Iterate over generator
            summarizedPage = summarizer(articleCombined,max_length=1000, min_length=200, do_sample=True)
            return summarizedPage[0]["summary_text"]
        except PdfReadError as pre: # Second check if file extension is ".pdf" format when first check fails.
            return f"Oops! a PDF Read Error {pre} happened. Please retry the task once the issue has been resolved."
        except OSError as os: # Exception handling of OS errors, as PdfReadError doesn't catch all file extension errors.
            return f"Oops! an OS Error {os} happened. Please retry the task once the issue has been resolved."
    else:
        return f"Oops! The task failed. Please retry the task once the issue has been resolved." # Message if program fails to execute

def pdfKE(pdfFile, language): # Function to extract keywords from the PDF, language input needed as language needs to be part of stopwords folder.
    if pdfFile.endswith(".pdf"): # First check if file extension is ".pdf" format.
        try:
            articleCombined = articleC(pdfFile).lower() # Iterate over generator
            tokens = word_tokenize(articleCombined) # Tokenize all words in the article
            punctuations = ["(",")",";",":","[","]",",","!","=","==","<",">","@","#","$","%","^","&","*",".","//","{","}","...","``","+","\'\'","-","~","\"","’",]
            stopWords = stopwords.words(f'{language}')
            keywords = [word for word in tokens if word not in stopWords and word not in punctuations] # Filter the words so that mostlikely keywords will be extracted
            keywordExtracted = pd.Series(keywords).value_counts().index[:5] # Keywords formatting as a list
            keywordString = " ".join(keywordExtracted) # Keywords formatting as a string
            keywordDict = {i+1:keywordExtracted[i] for i in range(5)}
            return keywordDict
        except PdfReadError as pre: # Second check if file extension is ".pdf" format when first check fails.
            return f"Oops! a PDF Read Error {pre} happened. Please retry the task once the issue has been resolved."
        except OSError as os: # Exception handling of OS errors, as PdfReadError doesn't catch all file extension errors.
            return f"Oops! an OS Error {os} happened. Please retry the task once the issue has been resolved."
    else:
        return f"Oops! The task failed. Please retry the task once the issue has been resolved." # Message if program fails to execute

In [84]:
# Summarize text
text = sumArticle2(ArticleToSummarize)
print(text)

This paper discusses several different types of models used to predict software development time and cost. The most commonly used is the Putnam model, which is based on a weighted average of various estimations made using standard methods such as logistic regression. However, the authors believe that it can be improved by using a more advanced statistical method called the "routinely weighted mean" which takes into account factors such as hours, days, and weeks. The authors suggest that this method can be extended to cover a wide range of industrial applications from manufacturing to space exploration to military planning to civil engineering. They claim that this new approach can predict future performance much more accurately than the previous methods. Another influential aspect of the study is the use of machine learning to predict real-time trends in the demand side of a business. For example, one of the main points in the paper is that the machine learning algorithms predicts futu

In [88]:
# Extract keywords and output them in string format
keywords = pdfKE(ArticleToSummarize, 'english')
print(keywords)

{1: 'model', 2: 'data', 3: 'putnam', 4: 'manpower', 5: 'gamma'}


In [86]:
# Summarization translation
translation = pipeline("translation", model="Helsinki-NLP/opus-mt-en-nl") # Model used for the translation, imported from Huggingface (https://huggingface.co/Helsinki-NLP/opus-mt-en-nl)
translatedText = translation(text)[0]['translation_text']
print(translatedText)

Dit document bespreekt verschillende soorten modellen die gebruikt worden om de ontwikkelingstijd en -kosten van software te voorspellen. De meest gebruikte is het Putnam-model, dat gebaseerd is op een gewogen gemiddelde van verschillende schattingen met behulp van standaardmethoden zoals logistieke regressie. Echter, de auteurs geloven dat het kan worden verbeterd door gebruik te maken van een meer geavanceerde statistische methode genaamd de "routinely gewogen gemiddelde" die rekening houdt met factoren zoals uren, dagen en weken. De auteurs suggereren dat deze methode kan worden uitgebreid tot een breed scala van industriële toepassingen, van productie tot ruimteverkenning tot militaire planning tot civiele techniek. Zij beweren dat deze nieuwe benadering toekomstige prestaties veel nauwkeuriger kan voorspellen dan de vorige methoden. Een ander invloedrijk aspect van de studie is het gebruik van machine learning om real-time trends in de vraagzijde van een bedrijf te voorspellen. Bi

In [28]:
# Cell only works in environment that works on pip. Evaluate isn't a library that has a conda recipe.

import evaluate

metric = evaluate.load("sacrebleu") # Model generated translation evaluated with DeepL generated translation

evaluationText = "In dit artikel worden verschillende soorten modellen besproken die worden gebruikt om de ontwikkelingstijd en -kosten van software te voorspellen. Het meest gebruikte model is het Putnam SLIM model, dat gebaseerd is op een hellings-leercurve. De auteurs zijn echter van mening dat dit model niet betrouwbaar is bij het voorspellen van zeer vroege stadia van een project. Ze gebruiken een andere benadering, het 'Putnam-model', om toekomstige prestaties te voorspellen. In dit artikel stellen ze een nieuw type model voor dat toekomstige prestaties veel nauwkeuriger voorspelt dan het vorige Putnam-model. Dit nieuwe model kan op verschillende manieren worden geïmplementeerd, waaronder: het correleren van statistieken van real-time experimenten met machine learning, het gebruik van statistische methoden zoals rinterpreter en correlatieanalyse, modellering door directe observatie en het gebruik van geavanceerde statistische technieken zoals gewogen gemiddelden. De auteurs bespreken ook verschillende benaderingen voor het voorspellen van toekomstige prestaties van een bepaald soort product of functie. Als een bedrijf bijvoorbeeld een nieuw stuk machine zou introduceren, zou het in staat moeten zijn om het snel en op een hoog niveau te produceren over een lange periode."

translatedText = "Het meest gebruikte model is het Putnam SLIM model, dat gebaseerd is op een hellings-learning curve. Echter, de auteurs geloven dat het niet betrouwbaar is in het voorspellen van zeer vroege stadia van een project. Ze gebruiken een andere aanpak genaamd het 'Putnam model' om toekomstige prestaties te voorspellen. In dit document suggereren ze een nieuw type model dat toekomstige prestaties veel nauwkeuriger voorspelt dan het vorige Putnam model. Dit nieuwe model kan worden toegepast op verschillende manieren, waaronder: het correleren van statistieken van real-time experimenten met machine learning, gebruik makend van statistische methoden zoals rinterpreter en correlatieanalyse, modelleren door middel van directe observatie, en gebruik makend van geavanceerde statistische technieken zoals gewogen gemiddelde. De auteurs bespreken ook verschillende benaderingen voor het voorspellen van toekomstige prestaties van een bepaald soort product of functie. Bijvoorbeeld, als een bedrijf een nieuw stuk machines zou moeten kunnen introduceren, zou het snel en op een hoog niveau van tijd moeten produceren."

predictions = [
    translatedText
]
references = [
    [
        evaluationText
    ]
]
metric.compute(predictions=predictions, references=references)

### ==============================
#{'score': 53.86425158362457,
#'counts': [149, 112, 95, 80],
#'totals': [173, 172, 171, 170],
#'precisions': [86.1271676300578,
# 65.11627906976744,
# 55.55555555555556,
# 47.05882352941177],
#'bp': 0.8704644809074915,
#'sys_len': 173,
#'ref_len': 197}
### ===============================

ImportError: DLL load failed while importing lib: Kan opgegeven procedure niet vinden.